In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../07_Embeddings/exp/result/07_v1_01/oof_df.csv',1), # svr
    
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    #('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    #('../../14_Baseline4/exp/result/14_v1_17/oof_df.csv',1), # deberta-v3-large, 10folds
    #('../../14_Baseline4/exp/result/14_v1_18/oof_df.csv',1), # deberta-large, 10folds
    
    #('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    #('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
    #('../../20_Distillation2/exp/result/20_v2_07/oof_df.csv',1), # deberta-v3-base <- svr
    #('../../20_Distillation2/exp/result/20_v2_08/oof_df.csv',1), # deberta-v3-base <- xlm-rb-b
    #('../../20_Distillation2/exp/result/20_v2_09/oof_df.csv',1), # deberta-v3-base <- xlm-rb-l
    #('../../20_Distillation2/exp/result/20_v2_10/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    #('../../20_Distillation2/exp/result/20_v2_11/oof_df.csv',1), # deberta-v3-base <- dbv3-b, 10folds
    
    # stats
    ('../../24_Statistics/exp/result/24_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../24_Statistics/exp/result/24_v1_02/oof_df.csv',1), # xlm-roberta-base
]

#tmp_df = pd.read_csv('../../14_Baseline4/exp/result/14_v1_17/oof_df.csv').dropna()
tmp_df = train_df.copy()
text_ids = tmp_df["text_id"].values

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
oof_df_list = [oof_df.loc[oof_df["text_id"].isin(text_ids)].reset_index(drop=True) for oof_df in oof_df_list]

num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
train_df = train_df.loc[train_df["text_id"].isin(text_ids)].reset_index(drop=True)

In [6]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [7]:
oof_df.shape, train_df.shape

((3911, 19), (3911, 8))

In [8]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df.loc[train_df["text_id"].isin(oof_df["text_id"]), TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4436


# Optimize class-wise

In [9]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(7, 3911, 6)

In [10]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [11]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0032
Function value obtained: 0.4721
Current minimum: 0.4721
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: 0.4715
Current minimum: 0.4715
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: 0.4738
Current minimum: 0.4715
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 0.4717
Current minimum: 0.4715
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.6554
Function value obtained: 0.4710
Current minimum: 0.4710
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.8046
Function value obtained: 0.4724
Current minimum: 0.4710
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.6722
Function value obtained: 0.4734
Current minimum: 0.4710
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.7471
Function value obtained: 0.4734
Current minimum: 0.4710
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.7424
Function value obtained: 0.4726
Current minimum: 0.4710
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.0625
Function value obtained: 0.4720
Current minimum: 0.4709
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.1483
Function value obtained: 0.4709
Current minimum: 0.4709
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.2293
Function value obtained: 0.4722
Current minimum: 0.4709
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.0949
Function value obtained: 0.4718
Current minimum: 0.4709
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.2435
Function value obtained: 0.4710
Current minimum: 0.4709
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.4276
Function value obtained: 0.4374
Current minimum: 0.4365
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.4210
Function value obtained: 0.4371
Current minimum: 0.4365
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.4009
Function value obtained: 0.4374
Current minimum: 0.4365
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3951
Function value obtained: 0.4372
Current minimum: 0.4365
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3691
Function value obtained: 0.4375
Current minimum: 0.4365
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.9067
Function value obtained: 0.4375
Current minimum: 0.4365
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.7922
Function value obtained: 0.4381
Current minimum: 0.4365
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.8539
Function value obtained: 0.4376
Current minimum: 0.4365
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.9813
Function value obtained: 0.4378
Current minimum: 0.4365
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.8734
Function value obtained: 0.4389
Current minimum: 0.4365
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.2871
Function value obtained: 0.4676
Current minimum: 0.4636
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.3232
Function value obtained: 0.4634
Current minimum: 0.4634
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.3113
Function value obtained: 0.4647
Current minimum: 0.4634
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3957
Function value obtained: 0.4649
Current minimum: 0.4634
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3547
Function value obtained: 0.4642
Current minimum: 0.4634
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.9380
Function value obtained: 0.4644
Current minimum: 0.4625
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.7508
Function value obtained: 0.4655
Current minimum: 0.4625
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.7181
Function value obtained: 0.4648
Current minimum: 0.4625
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.8356
Function value obtained: 0.4639
Current minimum: 0.4625
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.8528
Function value obtained: 0.4624
Current minimum: 0.4624
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 1.1439
Function value obtained: 0.4661
Current minimum: 0.4622
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 1.2345
Function value obtained: 0.4634
Current minimum: 0.4622
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 1.1806
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 1.3354
Function value obtained: 0.4654
Current minimum: 0.4622
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 1.3125
Function value obtained: 0.4623
Current minimum: 0.4622
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3789
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.4231
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.6610
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.6430
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.4857
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.9479
Function value obtained: 0.4371
Current minimum: 0.4367
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.8784
Function value obtained: 0.4375
Current minimum: 0.4367
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 1.0314
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 1.0204
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 1.0431
Function value obtained: 0.4370
Current minimum: 0.4367
Iteration No: 75 started. Sea

In [12]:
best_weights

{'cohesion': array([0.28476206, 0.14326502, 0.2640735 , 0.1101887 , 0.06709919,
        0.10213532, 0.02847621]),
 'syntax': array([0.21378197, 0.21190729, 0.24038503, 0.13358415, 0.0240385 ,
        0.15226456, 0.0240385 ]),
 'vocabulary': array([0.20180057, 0.27257072, 0.140511  , 0.02725707, 0.02725707,
        0.23512051, 0.09548305]),
 'phraseology': array([0.21490071, 0.19110625, 0.16711152, 0.13622574, 0.06315134,
        0.15607321, 0.07143123]),
 'grammar': array([0.23245322, 0.36741356, 0.03674136, 0.09999241, 0.12914483,
        0.09751328, 0.03674136]),
 'conventions': array([0.20827651, 0.14789305, 0.17185414, 0.13814674, 0.16056793,
        0.11270139, 0.06056025])}

In [13]:
scores

{'cohesion': 0.4709242248220708,
 'syntax': 0.43646747927065366,
 'vocabulary': 0.4045088915387113,
 'phraseology': 0.4456691880099001,
 'grammar': 0.4622307939550335,
 'conventions': 0.43666879130859476}

In [14]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4427


# Error Analysis - Check Corr

In [15]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [16]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.946098,0.912607,0.909612,0.823951,0.887180
syntax,0.946098,1.000000,0.936142,0.959632,0.916883,0.930415
vocabulary,0.912607,0.936142,1.000000,0.960207,0.875835,0.890906
phraseology,0.909612,0.959632,0.960207,1.000000,0.942143,0.883277
grammar,0.823951,0.916883,0.875835,0.942143,1.000000,0.858211
conventions,0.887180,0.930415,0.890906,0.883277,0.858211,1.000000
